In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../src/')
import utils as ut
import plotter
import wave as w
import pickle

### Computing the speed of the wave varying r0

In [2]:
def build_assay_wave(id_, r0, D, beta, gamma=1.0, cutoff=1):
    """
    It created a wave object with a good tuning of the parameters.
    The object has to be run.
    """
    
    dx = ut.lin_from_two_points(np.log(r0), np.log(0.01), 0.001, np.log(1), 0.006)
    dt = ut.dt_from_cfl(0.03, D, dx)

    p = w.Vwave_pars(id_, tot_time=400, dt=dt, dx=dx, 
                     n_x_bins=int( ut.lin_from_two_points(r0, 0.01, 3, 1, 50)/dx ), 
                     M=5, r0=r0, D_coef=D, beta=beta, alpha=0.0, gamma=gamma,
                     Nh=10**10, N0=10**8, cutoff=cutoff,
                     t_burn=ut.lin_from_two_points(r0, 0.01, 200, 1, 10000), 
                     t_burn_cutoff=ut.lin_from_two_points(r0, 0.01, 70, 1, 1000), 
                     back_width_fract=ut.lin_from_two_points(np.log(r0), np.log(0.0075), 0.1, np.log(1), 3),
                     traj_step=int(10/dt), check_step=0, traj_after_burn=True, verbose=True)
    return w.Vwave(p)

In [3]:
r0_list = np.logspace(np.log10(0.0075), np.log10(5), 20)
D = 1e-5
beta = 2

In [ ]:
assays = []
count = 0
for r0 in r0_list:
    assays.append(build_assay_wave(count, r0, D, beta))
    count += 1
    
assays = ut.multiprocess_sim(assays, nproc=4)

In [21]:
# saving the assays in a file 
#f = open('data/assays.pickle', 'wb')
#pickle.dump(assays, f)
#f.close()

In [5]:
# importing the assays
f = open('data/assays.pickle', 'rb')
assays = pickle.load(f)
f.close()

In [7]:
speeds = np.zeros(len(r0_list))
f_tips = np.zeros(len(r0_list))
s_tips = np.zeros(len(r0_list))
speed_lin = np.zeros(len(r0_list))
speed_FKPP = np.zeros(len(r0_list))

count = 0
for i, r in enumerate(r0_list):
    for a in assays:
        if a.p.id == count:
            speeds[i] = np.mean(a.traj.speed(10))
            f_tips[i] = np.mean(a.traj.f_tip)
            s_tips[i] = np.mean(a.traj.s_tip)
            speed_FKPP[i] = ut.speed_FKPP(a.p, 1e20, 1e-10) # No size correction here
            n = ut.size_lin(a.p, 1)
            speed_lin[i] = ut.speed_lin(a.p, n, 1)
    count+=1

In [11]:
f = open('data/det_speed.tsv', 'w')

header = '#r0\tspeed\tF_T\ts_T\tspeed_FKPP\tspeed_linfit\n'
f.write(header)

for i in range(len(r0_list)):
    f.write(str(r0_list[i]) + '\t')
    f.write(str(speeds[i]) + '\t')
    f.write(str(f_tips[i]) + '\t')
    f.write(str(s_tips[i]) + '\t')
    f.write(str(speed_FKPP[i]) + '\t')
    f.write(str(speed_lin[i]) + '\t')
    f.write('\n')
    
f.close()

In [20]:
f = open('data/det_speed_pars.tsv', 'w')

header = '#parameter\tvalue\n'
f.write(header)

f.write('D\t' + str(D) + '\n')
f.write('beta\t' + str(beta) + '\n')
f.write('alpha\t' + str(assays[0].p.alpha) + '\n')
f.write('Nh\t' + str(assays[0].p.Nh) + '\n')
f.write('M\t' + str(assays[0].p.M) + '\n')
f.write('cutoff\t' + str(assays[0].p.cutoff) + '\n')

f.close()

## Speed re-scaling

In [18]:
r0_list_resc = np.logspace(np.log10(0.01), np.log10(2), 10)
Ds = [1e-5, 0.5e-5]
betas = [2, 4]

In [ ]:
assay_resc = []
count = 0
for r0 in r0_list_resc:
    for D in Ds:
        for b in betas:
            assay_resc.append(build_assay_wave(count, r0, D, b))
            count += 1
    
assay_resc = ut.multiprocess_sim(assay_resc, nproc=4)

In [22]:
# saving the assays in a file 
f = open('data/assays_resc.pickle', 'wb')
pickle.dump(assay_resc, f)
f.close()

In [23]:
# importing the assays
f = open('data/assays_resc.pickle', 'rb')
assay_resc = pickle.load(f)
f.close()

In [24]:
speeds_resc = np.zeros((len(r0_list_resc),len(Ds),len(betas)))
f_tips_resc = np.zeros((len(r0_list_resc),len(Ds),len(betas)))
s_tips_resc = np.zeros((len(r0_list_resc),len(Ds),len(betas)))
speed_lin_resc = np.zeros(len(r0_list_resc))
speed_FKPP_resc = np.zeros(len(r0_list_resc))

count = 0
for i,r0 in enumerate(r0_list_resc):
    
    for j, D in enumerate(Ds):
        for k, b in enumerate(betas):
            for a in assay_resc:
                if a.p.id == count:
                    #print(a.p.id, a.p.r0, r0, a.p.D_coef, D)
                    speeds_resc[i,j,k] = np.mean(a.traj.speed(10))
                    f_tips_resc[i,j,k] = np.mean(a.traj.f_tip)
                    s_tips_resc[i,j,k] = np.mean(a.traj.s_tip)
                    
                    if j == 0 and k == 0:
                        speed_FKPP_resc[i] = ut.speed_FKPP(a.p, 1e20, 1e-10) # No size correction here
                    if j == 1 and k == 1:
                        n = ut.size_lin(a.p, 1)
                        speed_lin_resc[i] = ut.speed_lin(a.p, n, 1)
            
            count+=1

In [33]:
f = open('data/rescaling_speed.tsv', 'w')

header = '#run\tr0\tspeed\tF_T\ts_T\tspeed_FKPP\tspeed_linfit\n'
f.write(header)

run = 0
for j in range(len(Ds)):
    for k in range(len(betas)):
        for i in range(len(r0_list_resc)):
            f.write(str(run) + '\t')
            f.write(str(r0_list_resc[i]) + '\t')
            f.write(str(speeds_resc[i,j,k]) + '\t')
            f.write(str(f_tips_resc[i,j,k]) + '\t')
            f.write(str(s_tips_resc[i,j,k]) + '\t')
            if j == 0 and k == 0:
                f.write(str(speed_FKPP_resc[i]) + '\t')
            else:
                f.write('\t')
            if j == 1 and k == 1:
                f.write(str(speed_lin_resc[i]) + '\t')
            else:
                f.write('\t')
            f.write('\n')
        run += 1

f.close()

In [29]:
f = open('data/rescaling_speed_pars.tsv', 'w')

header = '#parameter\tvalue\trun\n'
f.write(header)

run = 0
for i in range(len(Ds)):
    for j in range(len(betas)):
        for a in assay_resc:
            if a.p.id == run:
                f.write('D\t' + str(a.p.D_coef) + '\t' + str(run) + '\n')
                f.write('beta\t' + str(a.p.beta) + '\t' + str(run) + '\n')
                f.write('alpha\t' + str(a.p.alpha) + '\t' + str(run) + '\n')
                f.write('Nh\t' + str(a.p.Nh) + '\t' + str(run) + '\n')
                f.write('M\t' + str(a.p.M) + '\t' + str(run) + '\n')
                f.write('cutoff\t' + str(a.p.cutoff) + '\t' + str(run) + '\n')
        run += 1

f.close()

## Computing the speed at different cutoffs

In [34]:
r0_list_cutoff = np.logspace(np.log10(0.0075), np.log10(2), 15)
D_cutoff = 1e-5
beta_cutoff = 2
cutoffs = np.logspace(np.log10(0.001),np.log10(1000),3)

In [ ]:
assays_cutoff = []
count = 0
for r0 in r0_list_cutoff:
    for c in cutoffs:
        assays_cutoff.append(build_assay_wave(count, r0, D_cutoff, beta_cutoff, cutoff=c))
        count += 1
    
assays_cutoff = ut.multiprocess_sim(assays_cutoff, nproc=4)

In [36]:
# saving the assays in a file 
#f = open('data/assays_cutoff.pickle', 'wb')
#pickle.dump(assays_cutoff, f)
#f.close()

In [37]:
# importing the assays
f = open('data/assays_cutoff.pickle', 'rb')
assays_cutoff = pickle.load(f)
f.close()

In [47]:
speeds_cutoff = np.zeros((len(r0_list_cutoff), len(cutoffs)))
f_tips_cutoff = np.zeros((len(r0_list_cutoff), len(cutoffs)))
s_tips_cutoff = np.zeros((len(r0_list_cutoff), len(cutoffs)))
speeds_lin_cutoff = np.zeros((len(r0_list_cutoff), len(cutoffs)))
speeds_FKPP_cutoff = np.zeros((len(r0_list_cutoff), len(cutoffs)))
count = 0
for i, r in enumerate(r0_list_cutoff):
    for j, c in enumerate(cutoffs):
        for a in assays_cutoff:
            if a.p.id == count:
                speeds_cutoff[i,j] = np.mean(a.traj.speed())
                f_tips_cutoff[i,j] = np.mean(a.traj.f_tip)
                s_tips_cutoff[i,j] = np.mean(a.traj.s_tip)
                speeds_FKPP_cutoff[i,j] = ut.speed_FKPP(a.p, 1e20, 1e-10)
                n = ut.size_lin(a.p, c)
                speeds_lin_cutoff[i,j] = ut.speed_lin(a.p, n, c)
        count+=1

In [52]:
f = open('data/cutoff_speed.tsv', 'w')

header = '#run\tr0\tspeed\tF_T\ts_T\tspeed_FKPP\tspeed_linfit\n'
f.write(header)

run = 0
for j in range(len(cutoffs)):
    for i in range(len(r0_list_cutoff)):
        f.write(str(run) + '\t')
        f.write(str(r0_list_cutoff[i]) + '\t')
        f.write(str(speeds_cutoff[i,j]) + '\t')
        f.write(str(f_tips_cutoff[i,j]) + '\t')
        f.write(str(s_tips_cutoff[i,j]) + '\t')
        f.write(str(speeds_FKPP_cutoff[i,j]) + '\t')
        f.write(str(speeds_lin_cutoff[i,j]) + '\t')
        f.write('\n')
    run += 1

f.close()

In [51]:
f = open('data/cutoff_speed_pars.tsv', 'w')

header = '#parameter\tvalue\trun\n'
f.write(header)

run = 0
for j in range(len(cutoffs)):
    for a in assays_cutoff:
        if a.p.id == run:
            f.write('D\t' + str(a.p.D_coef) + '\t' + str(run) + '\n')
            f.write('beta\t' + str(a.p.beta) + '\t' + str(run) + '\n')
            f.write('alpha\t' + str(a.p.alpha) + '\t' + str(run) + '\n')
            f.write('Nh\t' + str(a.p.Nh) + '\t' + str(run) + '\n')
            f.write('M\t' + str(a.p.M) + '\t' + str(run) + '\n')
            f.write('cutoff\t' + str(a.p.cutoff) + '\t' + str(run) + '\n')
    run += 1

f.close()

## Computing the speed at different delta x

In [3]:
def build_assay_wave_dx(id_, r0, D, beta, Delta_x=0.5, gamma=1.0, cutoff=1):
    dx = ut.lin_from_two_points(np.log(r0/Delta_x), np.log(0.01), 0.001, np.log(1), 0.005)
    dt = ut.dt_from_cfl(0.03, D, dx)

    p = w.Vwave_pars(id_, tot_time=2000, dt=dt, dx=dx, 
                     n_x_bins=int( ut.lin_from_two_points(r0/Delta_x, 0.01, 3, 1, 50)/dx ), 
                     M=5, r0=r0, D_coef=D, beta=beta, alpha=0.0, gamma=gamma,
                     Nh=10**12, N0=10**10, cutoff=cutoff,
                     t_burn=ut.lin_from_two_points(r0/Delta_x, 0.01, 1000, 1, 15000), 
                     t_burn_cutoff=ut.lin_from_two_points(r0/Delta_x, 0.01, 70, 1, 1000), 
                     back_width_fract=ut.lin_from_two_points(np.log(r0/Delta_x), np.log(0.0075), 0.1, np.log(1), 3),
                     traj_step=int(10/dt), check_step=int(0/dt), traj_after_burn=True, verbose=True)
    return w.Vwave(p)

In [4]:
delta_xs = [0.5, 1, 2]
c_list_dx = np.logspace(np.log10(0.0075), np.log10(2), 15)
D0 = 1e-5
beta = 2

In [ ]:
assays_dx = []
count = 0
for c in c_list_dx:
    for dx in delta_xs:
        assays_dx.append(build_assay_wave_dx(count, c * dx, D0 * dx**2, beta, Delta_x=dx))
        count += 1
    
assays_dx = ut.multiprocess_sim(assays_dx, nproc=4)

proc. 0 WARNING: dx is not small compared to r0. dx/r0= 0.2000326738089065
proc. 1 WARNING: dx is not small compared to r0. dx/r0= 0.10001633690445325
proc. 3 WARNING: dx is not small compared to r0. dx/r0= 0.19623159428464632
proc. 6 WARNING: dx is not small compared to r0. dx/r0= 0.17327873854697018
proc. 9 WARNING: dx is not small compared to r0. dx/r0= 0.14418778632441664
proc. 12 WARNING: dx is not small compared to r0. dx/r0= 0.11548229901699457
  

Process 1:   0%|          | 0/1756253 [00:00<?, ?it/s]

Process 3:   0%|          | 0/209014 [00:00<?, ?it/s]

Process 2:   0%|          | 0/7025014 [00:00<?, ?it/s]

Process 0:   0%|          | 0/439063 [00:00<?, ?it/s]

Process 4:   0%|          | 0/836058 [00:00<?, ?it/s]

Process 5:   0%|          | 0/3344232 [00:00<?, ?it/s]

Process 6:   0%|          | 0/123786 [00:00<?, ?it/s]

Process 7:   0%|          | 0/495146 [00:00<?, ?it/s]

In [ ]:
# saving the assays in a file 
f = open('data/assays_dx.pickle', 'wb')
pickle.dump(assays_dx, f)
f.close()

In [ ]:
# importing the assays
f = open('data/assays_dx.pickle', 'rb')
assays_dx = pickle.load(f)
f.close()

In [ ]:
speeds_dx = np.zeros((len(c_list_dx), len(delta_xs)))
count = 0
for i, r in enumerate(c_list_dx):
    for j in range(len(delta_xs)):
        for a in assays_dx:
            if a.p.id == count:
                speeds_dx[i,j] = np.mean(a.traj.speed(10))
        count+=1

In [ ]:
f = open('data/dx_speed.tsv', 'w')

header = '#run\tr0\tspeed\tF_T\ts_T\tspeed_FKPP\tspeed_linfit\n'
f.write(header)

run = 0
for j in range(len(delta_xs)):
    for i in range(len(c_list_dx)):
        f.write(str(run) + '\t')
        f.write(str(c_list_dx[i]*dx) + '\t')
        f.write(str(speeds_dx[i,j]) + '\t')
        f.write('\n')
    run += 1

f.close()